In [451]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests

from time import sleep
import pandas as pd 
import numpy as np
import re
import tldextract
import html
import scrape_schema_recipe
from ast import literal_eval
from html import unescape


In [1031]:
data = pd.read_csv ('/Users/vincentsalamand/Documents/Clubmama/Recipes/Raw/recipes_undejeunerdesoleil.csv')
data = data.loc[:, data.columns.isin(['name', 'author', 'datePublished', 'recipeIngredient', 'recipeInstructions', 'url', 'image', 'prepTime', 'cookTime', 'totalTime', 'recipeYield'])]



In [1032]:

data = data[data['name'].notna()]
data = data[data['image'].notna()]
data = data[data['recipeIngredient'].notna()]
data = data[data['recipeInstructions'].notna()]

data


,name,author,datePublished,image,recipeYield,prepTime,cookTime,recipeIngredient,recipeInstructions,url,totalTime
0,Morue à la vénitienne (baccalà alla vicentina),"{'@type': 'Person', 'name': 'Edda Onorato'}",2020-02-20 08:00:42+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,6 personnes,0 days 00:20:00.000000000,0 days 03:00:00.000000000,['1 kg morue (dessalée entière ou en gros morc...,"[{'@type': 'HowToStep', 'text': 'Eplucher puis...",https://www.undejeunerdesoleil.com/2020/02/mor...,NaN
1,Joue de boeuf mijotée à la bière aux champigno...,"{'@type': 'Person', 'name': 'Edda Onorato'}",2020-02-16 08:00:21+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,6 personnes,0 days 00:20:00.000000000,0 days 02:00:00.000000000,['1 kg boeuf (joue (parées et coupées en gros ...,"[{'@type': 'HowToStep', 'text': 'Dans la cocot...",https://www.undejeunerdesoleil.com/2020/02/jou...,NaN
2,Gratin de crêpes ricotta épinards comme à Flor...,"{'@type': 'Person', 'name': 'Edda Onorato'}",2020-02-07 08:00:07+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,6 personnes,0 days 00:40:00.000000000,0 days 00:30:00.000000000,"['25 cl lait (frais entier)', '3 oeufs', '70 ...","[{'@type': 'HowToStep', 'text': 'Préparer la p...",https://www.undejeunerdesoleil.com/2020/02/gra...,NaN
3,Lasagnes façon bolognaise merveilleuse recette...,"{'@type': 'Person', 'name': 'Edda Onorato'}",2020-02-03 08:00:03+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,10 personnes,0 days 01:00:00.000000000,0 days 02:00:00.000000000,['600 g boeuf (haché à 15% de matière grasse)'...,"[{'@type': 'HowToStep', 'text': 'Préparer la s...",https://www.undejeunerdesoleil.com/2020/02/las...,NaN
4,Coquelet rôti au vinaigre balsamique,"{'@type': 'Person', 'name': 'Edda Onorato'}",2020-01-18 08:00:45+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,4 personnes,0 days 00:10:00.000000000,0 days 01:00:00.000000000,"['2 coquelets (de 500 g chacuns )', '5 cl vin...","[{'@type': 'HowToStep', 'text': 'Préchauffer l...",https://www.undejeunerdesoleil.com/2020/01/coq...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
442,Cocktail Balalaïka ou Troïka,"{'@type': 'Person', 'name': 'Edda Onorato'}",2018-12-31 08:00:23+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,4 personnes,0 days 00:10:00.000000000,NaN,"['12 cl vodka', '8 cl triple sec (type Grand M...","[{'@type': 'HowToStep', 'text': ""Verser le suc...",https://www.undejeunerdesoleil.com/2018/12/coc...,NaN
443,Gin Tonic,"{'@type': 'Person', 'name': 'Edda Onorato'}",2018-09-29 09:00:26+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,4 personnes,0 days 00:05:00.000000000,NaN,"['16 cl gin', '8 cl jus de citron (facultatif)...","[{'@type': 'HowToStep', 'text': 'Remplir la mo...",https://www.undejeunerdesoleil.com/2018/09/gin...,0 days 00:05:00.000000000
444,Bellini cocktail,"{'@type': 'Person', 'name': 'Edda'}",2018-09-08 09:00:22+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,4 personnes,0 days 00:10:00.000000000,NaN,['160 g pêche (blanche (pulpe) ou de nectarine...,"[{'@type': 'HowToStep', 'text': 'Mettre les ve...",https://www.undejeunerdesoleil.com/2018/09/bel...,0 days 00:10:00.000000000
445,Irish coffee,"{'@type': 'Person', 'name': 'Edda Onorato'}",2018-02-26 09:00:37+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,4 personnes,0 days 00:10:00.000000000,0 days 00:05:00.000000000,"['16 cl whisky (Irish)', '32 cl café (chaud)',...","[{'@type': 'HowToStep', 'text': ""Préparer la c...",https://www.undejeunerdesoleil.com/2018/02/iri...,NaN


In [1038]:
#data["prepTime"] = pd.to_timedelta(data.iloc[:, 2])
#data["cookTime"] = pd.to_timedelta(data.iloc[:, 6])
#data["totalTime"] = pd.to_timedelta(data.iloc[:, 7])
#data.author = data.author.apply(lambda x: literal_eval(x)['name'] if str(literal_eval(x)).startswith('{') else x)
data.recipeInstructions = data.recipeInstructions.apply(lambda x : str([item for sublist in [key['itemListElement'] for key in literal_eval(x)] for item in sublist] if "itemListElement" in literal_eval(x)[0] else x ))
data.recipeInstructions = data.recipeInstructions.apply(lambda x : str([key['text'] for key in literal_eval(x) if 'text' in key] if str(literal_eval(x)[0]).startswith('{') else x))
#data.recipeInstructions = data.recipeInstructions.apply(lambda x : literal_eval(unescape(x)) if str(x).startswith('[') else [unescape(str(x).replace('\n', ''))])
data.recipeInstructions = data.recipeInstructions.apply(lambda x : unescape(x))
#data.recipeIngredient = data.recipeIngredient.apply(lambda x : [unescape(string).replace('\r', '') for string in literal_eval(x)] )
#data.image = data.image.apply(lambda x : x if x.startswith('[') else x.split() )
#data.name = data.name.apply(lambda x : unescape(x))
#data.image = data.image.apply(lambda x : [el for el in x if el.startswith('http')])
#data.image = data.image.apply(lambda x : x.split())



In [874]:
data.at[163, 'image'] = ['https://lh3.googleusercontent.com/-1m8LeLqC6yc/UqD_JJ3-YjI/AAAAAAAAD_k/3S6UP1l_iiE/s800/3.jpg']
data.loc[163].url


'http://emiliemurmure.com/detox-salade-de-kale-patate-douce-grenade-a-la-sauce-tahini/'

In [1034]:
data[data['image'].isnull()]


,name,author,datePublished,image,recipeYield,prepTime,cookTime,recipeIngredient,recipeInstructions,url,totalTime


In [1040]:
data.to_csv (r'/Users/vincentsalamand/Documents/Clubmama/Recipes/undejeunerdesoleil.csv', index = None, header=True)


In [1039]:
data

,name,author,datePublished,image,recipeYield,prepTime,cookTime,recipeIngredient,recipeInstructions,url,totalTime
0,Morue à la vénitienne (baccalà alla vicentina),Edda Onorato,2020-02-20 08:00:42+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,6 personnes,0 days 00:20:00.000000000,0 days 03:00:00.000000000,['1 kg morue (dessalée entière ou en gros morc...,['Eplucher puis émincer les oignons. Chauffer ...,https://www.undejeunerdesoleil.com/2020/02/mor...,NaN
1,Joue de boeuf mijotée à la bière aux champigno...,Edda Onorato,2020-02-16 08:00:21+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,6 personnes,0 days 00:20:00.000000000,0 days 02:00:00.000000000,['1 kg boeuf (joue (parées et coupées en gros ...,['Dans la cocotte de base de la mijoteuse (j'a...,https://www.undejeunerdesoleil.com/2020/02/jou...,NaN
2,Gratin de crêpes ricotta épinards comme à Flor...,Edda Onorato,2020-02-07 08:00:07+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,6 personnes,0 days 00:40:00.000000000,0 days 00:30:00.000000000,"['25 cl lait (frais entier)', '3 oeufs', '70 ...",['Préparer la pâte à crêpes. Dans le bol du mi...,https://www.undejeunerdesoleil.com/2020/02/gra...,NaN
3,Lasagnes façon bolognaise merveilleuse recette...,Edda Onorato,2020-02-03 08:00:03+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,10 personnes,0 days 01:00:00.000000000,0 days 02:00:00.000000000,['600 g boeuf (haché à 15% de matière grasse)'...,['Préparer la sauce (même la veille ou l'avant...,https://www.undejeunerdesoleil.com/2020/02/las...,NaN
4,Coquelet rôti au vinaigre balsamique,Edda Onorato,2020-01-18 08:00:45+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,4 personnes,0 days 00:10:00.000000000,0 days 01:00:00.000000000,"['2 coquelets (de 500 g chacuns )', '5 cl vin...",['Préchauffer le four à 190°C. Poser un papier...,https://www.undejeunerdesoleil.com/2020/01/coq...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
442,Cocktail Balalaïka ou Troïka,Edda Onorato,2018-12-31 08:00:23+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,4 personnes,0 days 00:10:00.000000000,NaN,"['12 cl vodka', '8 cl triple sec (type Grand M...","[""Verser le sucre dans une petite assiette. Mo...",https://www.undejeunerdesoleil.com/2018/12/coc...,NaN
443,Gin Tonic,Edda Onorato,2018-09-29 09:00:26+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,4 personnes,0 days 00:05:00.000000000,NaN,"['16 cl gin', '8 cl jus de citron (facultatif)...",['Remplir la moitié des verres avec des glaçon...,https://www.undejeunerdesoleil.com/2018/09/gin...,0 days 00:05:00.000000000
444,Bellini cocktail,Edda,2018-09-08 09:00:22+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,4 personnes,0 days 00:10:00.000000000,NaN,['160 g pêche (blanche (pulpe) ou de nectarine...,['Mettre les verres au frais 5 minutes. Pendan...,https://www.undejeunerdesoleil.com/2018/09/bel...,0 days 00:10:00.000000000
445,Irish coffee,Edda Onorato,2018-02-26 09:00:37+00:00,['https://www.undejeunerdesoleil.com/wp-conten...,4 personnes,0 days 00:10:00.000000000,0 days 00:05:00.000000000,"['16 cl whisky (Irish)', '32 cl café (chaud)',...","[""Préparer la crème fouettée : dans un bol, fo...",https://www.undejeunerdesoleil.com/2018/02/iri...,NaN


In [1001]:
data.iloc[0].image

['https://i2.wp.com/www.safrangourmand.fr/wp-content/uploads/2019/02/recette-bugnes3-christophe-felder.jpg?fit=800%2C534&ssl=1']